In [1]:
import os
import time
import re
import chromedriver_autoinstaller
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm

In [3]:
gu_list = ['마포구','서대문구','은평구','종로구','중구','용산구','성동구','광진구',
           '동대문구','성북구','강북구','도봉구','노원구','중랑구','강동구','송파구',
           '강남구','서초구','관악구','동작구','영등포구','금천구','구로구','양천구','강서구']

In [ ]:
cnt = 0
for index, gu_name in tqdm(enumerate(gu_list)):
    cnt += 1
    fileName = f'{gu_name}_{cnt}.csv' # index.__str__() + '_' + gu_name + '.'+'csv'
    file = open(fileName, 'w', encoding='utf-8')
    file.write("카페명" + "|" + "주소" + "|" + "영업시간" + "|" + "전화번호" + "|" + "대표사진주소" + "\n")
    file.close()                                    # 처음에 csv파일에 칼럼명 만들어주기
    # 크롬 버젼 확인 후 새로운 드라이버 설치해서 실행
    chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito") # 시크릿 모드
    chrome_options.add_argument("--headless") # 리눅스 GUI 없는 디스플레이 모드
    chrome_options.add_argument("--no-sandbox") # 리소스에 대한 엑서스 방지
    chrome_options.add_argument("--disable-setuid-sandbox") # 크롬 충돌 방지
    chrome_options.add_argument("--disable-dev-shm-usage") # 메모리 부족 에러 방지
    chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

    try: # 크롬 드라이버
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)   
    except:
        chromedriver_autoinstaller.install(True)
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)

    # WebDruverException Error 방지 기존의 드라이버 버젼으로 지정
    # driver = webdriver.Chrome(executable_path='/Users/cmblir/Python/Musinsa-Analysis/100/chromedriver')
    driver.implicitly_wait(10) # 10초정도 멈추기
    
    driver.get('https://map.kakao.com/')  # 주소 가져오기
    search_area = driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]') # 검색 창
    search_area.send_keys(gu_name + ' 카페')  # 검색어 입력
    driver.find_element(By.XPATH, '//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    driver.implicitly_wait(3) # 기다려 주자
    more_page = driver.find_element(By.ID, "info.search.place.more")
    # more_page.click()
    more_page.send_keys(Keys.ENTER) # 더보기 누르고
    # 첫 번째 검색 페이지 끝
    # driver.implicitly_wait(5) # 기다려 주자
    time.sleep(1)

    # next 사용 가능?
    next_btn = driver.find_element(By.ID, "info.search.page.next")
    has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
    Page = 1
    while has_next: # 다음 페이지가 있으면 loop
    # for i in range(2, 6): # 2, 3, 4, 5
        file = open(fileName, 'a', encoding='utf-8')
        time.sleep(1)
        # place_lists = driver.find_element(By.CSS_SELECTOR, ('#info\.search\.place\.list > li:nth-child(1)')
        # 페이지 루프
        #info\.search\.page\.no1 ~ .no5
        page_links = driver.find_elements(By.CSS_SELECTOR, "#info\.search\.page a")
        pages = [link for link in page_links if "HIDDEN" not in link.get_attribute("class").split(" ")]
        # print(len(pages), "개의 페이지 있음")
        # pages를 하나씩 클릭하면서
        for i in range(1, 6):
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            try:
                page = driver.find_element(By.XPATH, xPath)
                page.send_keys(Keys.ENTER)
            except ElementNotInteractableException:
                print('End of Page')
                break;
            sleep(3)
            place_lists = driver.find_elements(By.CSS_SELECTOR, '#info\.search\.place\.list > li')
            for p in place_lists: # WebElement
                # print(p.get_attribute('innerHTML'))
                # print("type of p:", type(p))
                store_html = p.get_attribute('innerHTML')
                store_info = BeautifulSoup(store_html, "html.parser")
                # BS -> 분석
                #
                place_name = store_info.select('.head_item > .tit_name > .link_name')
                # place_address = store_info.select('.info_item > .addr > p')
                # place_hour = store_info.select('.info_item > .openhour > p > a')
                # place_tel = store_info.select('.info_item > .contact > span')
                 # print("length:", len(place_name))
                if len(place_name) == 0:
                    continue # 광고
                place_name = store_info.select('.head_item > .tit_name > .link_name')[0].text
                place_address = store_info.select('.info_item > .addr > p')[0].text
                place_hour = store_info.select('.info_item > .openhour > p > a')[0].text
                place_tel = store_info.select('.info_item > .contact > span')[0].text


                # 사진url 수집
                detail = p.find_element(By.CSS_SELECTOR, 'div.info_item > div.contact > a.moreview')
                detail.send_keys(Keys.ENTER)

                driver.switch_to.window(driver.window_handles[-1])

                place_photo = ""
                try:
                    photo = driver.find_elements(By.CSS_SELECTOR, 'span.bg_present')
                    photo_url = photo.get_attribute('style')
                    m = re.search('"(.+?)"', photo_url)
                    if m:
                        place_photo = m.group(1)
                    else:
                        place_photo = ""
                except:
                    place_photo = ""
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                print(place_name, place_photo)

                file.write(place_name + "|" + place_address + "|" + place_hour + "|" + place_tel + "|" + place_photo + "\n")
            print(i, ' of', ' [ ' , Page, ' ] ')
        next_btn = driver.find_element(By.ID, "info.search.page.next")
        has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
        if not has_next:
            print('Arrow is Disabled')
            driver.close()
            file.close()
            break # 다음 페이지 없으니까 종료
        else: # 다음 페이지 있으면
            Page += 1
            next_btn.send_keys(Keys.ENTER)
    print("End of Crawl")

In [4]:
cnt = 100
for index, gu_name in tqdm(enumerate(gu_list)):
    cnt += 1
    fileName = f'{gu_name}_{cnt}.csv' # index.__str__() + '_' + gu_name + '.'+'csv'
    file = open(fileName, 'w', encoding='utf-8')
    file.write("맛집명" + "|" + "주소" + "|" + "영업시간" + "|" + "전화번호" + "|" + "대표사진주소" + "\n")
    file.close()                                    # 처음에 csv파일에 칼럼명 만들어주기
    # 크롬 버젼 확인 후 새로운 드라이버 설치해서 실행
    chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito") # 시크릿 모드
    chrome_options.add_argument("--headless") # 리눅스 GUI 없는 디스플레이 모드
    chrome_options.add_argument("--no-sandbox") # 리소스에 대한 엑서스 방지
    chrome_options.add_argument("--disable-setuid-sandbox") # 크롬 충돌 방지
    chrome_options.add_argument("--disable-dev-shm-usage") # 메모리 부족 에러 방지
    chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

    try: # 크롬 드라이버
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)   
    except:
        chromedriver_autoinstaller.install(True)
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver', chrome_options=chrome_options)

    # WebDruverException Error 방지 기존의 드라이버 버젼으로 지정
    # driver = webdriver.Chrome(executable_path='/Users/cmblir/Python/Musinsa-Analysis/100/chromedriver')
    driver.implicitly_wait(10) # 10초정도 멈추기
    
    driver.get('https://map.kakao.com/')  # 주소 가져오기
    search_area = driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]') # 검색 창
    search_area.send_keys(gu_name + ' 맛집')  # 검색어 입력
    driver.find_element(By.XPATH, '//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # Enter로 검색
    driver.implicitly_wait(3) # 기다려 주자
    more_page = driver.find_element(By.ID, "info.search.place.more")
    # more_page.click()
    more_page.send_keys(Keys.ENTER) # 더보기 누르고
    # 첫 번째 검색 페이지 끝
    # driver.implicitly_wait(5) # 기다려 주자
    time.sleep(1)

    # next 사용 가능?
    next_btn = driver.find_element(By.ID, "info.search.page.next")
    has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
    Page = 1
    while has_next: # 다음 페이지가 있으면 loop
    # for i in range(2, 6): # 2, 3, 4, 5
        file = open(fileName, 'a', encoding='utf-8')
        time.sleep(1)
        # place_lists = driver.find_element(By.CSS_SELECTOR, ('#info\.search\.place\.list > li:nth-child(1)')
        # 페이지 루프
        #info\.search\.page\.no1 ~ .no5
        page_links = driver.find_elements(By.CSS_SELECTOR, "#info\.search\.page a")
        pages = [link for link in page_links if "HIDDEN" not in link.get_attribute("class").split(" ")]
        # print(len(pages), "개의 페이지 있음")
        # pages를 하나씩 클릭하면서
        for i in range(1, 6):
            xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
            try:
                page = driver.find_element(By.XPATH, xPath)
                page.send_keys(Keys.ENTER)
            except ElementNotInteractableException:
                print('End of Page')
                break;
            sleep(3)
            place_lists = driver.find_elements(By.CSS_SELECTOR, '#info\.search\.place\.list > li')
            for p in place_lists: # WebElement
                # print(p.get_attribute('innerHTML'))
                # print("type of p:", type(p))
                store_html = p.get_attribute('innerHTML')
                store_info = BeautifulSoup(store_html, "html.parser")
                # BS -> 분석
                #
                place_name = store_info.select('.head_item > .tit_name > .link_name')
                # place_address = store_info.select('.info_item > .addr > p')
                # place_hour = store_info.select('.info_item > .openhour > p > a')
                # place_tel = store_info.select('.info_item > .contact > span')
                 # print("length:", len(place_name))
                if len(place_name) == 0:
                    continue # 광고
                place_name = store_info.select('.head_item > .tit_name > .link_name')[0].text
                place_address = store_info.select('.info_item > .addr > p')[0].text
                place_hour = store_info.select('.info_item > .openhour > p > a')[0].text
                place_tel = store_info.select('.info_item > .contact > span')[0].text


                # 사진url 수집
                detail = p.find_element(By.CSS_SELECTOR, 'div.info_item > div.contact > a.moreview')
                detail.send_keys(Keys.ENTER)

                driver.switch_to.window(driver.window_handles[-1])

                place_photo = ""
                try:
                    photo = driver.find_elements(By.CSS_SELECTOR, 'span.bg_present')
                    photo_url = photo.get_attribute('style')
                    m = re.search('"(.+?)"', photo_url)
                    if m:
                        place_photo = m.group(1)
                    else:
                        place_photo = ""
                except:
                    place_photo = ""
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
                print(place_name, place_photo)

                file.write(place_name + "|" + place_address + "|" + place_hour + "|" + place_tel + "|" + place_photo + "\n")
            print(i, ' of', ' [ ' , Page, ' ] ')
        next_btn = driver.find_element(By.ID, "info.search.page.next")
        has_next = "disabled" not in next_btn.get_attribute("class").split(" ")
        if not has_next:
            print('Arrow is Disabled')
            driver.close()
            file.close()
            break # 다음 페이지 없으니까 종료
        else: # 다음 페이지 있으면
            Page += 1
            next_btn.send_keys(Keys.ENTER)
    print("End of Crawl")

어반플랜트 합정 
피오니 홍대점 
바다회사랑 
프릳츠 도화점 
소이연남 
을밀대 본점 
크레이지카츠 
바다회사랑 2호점 
테이스티버거 
어글리베이커리 
황금콩밭 
망원동즉석우동전문돈까스 본점 
역전회관 
오레노라멘 합정본점 
마포진짜원조최대포 본점 
1  of  [  1  ] 
진미식당 
몽중헌 공덕점 
미쁘동 
마포나루 도화본점 
감성타코 홍대점 
하이디라오 홍대지점 
평이담백뼈칼국수 
원조조박집 본관 
헤키 
스케줄 합정 
943킹스크로스 
마포양지설렁탕 
곱 마포점 
카페공명 
연교 
2  of  [  1  ] 
진짜파스타 
오츠커피 마포점 
모연 
카페 레이어드 연남점 
우리바다수산 
우동카덴 
연남토마 본점 
소금집델리 망원 
아소정 
리치몬드과자점 성산본점 
중화복춘 
비파티세리 공덕점 
마포옥 
진진 본관 
우와 홍대본점 
3  of  [  1  ] 
두껍삼 마포직영점 
앤트러사이트 합정점 
청어람 망원점 
포비 합정점 
맛이차이나 
각시보쌈 
진대감 마포점 
또보겠지떡볶이집 호호시스터점 
마포원조떡볶이 
앤트러사이트 서교점 
카와카츠 
정돈 
툭툭누들타이 
테일러커피 서교점 
조선시대 
4  of  [  1  ] 
한강껍데기 
감나무집기사식당 
산울림1992 
당인리책발전소 
최강금돈까스 
무신사테라스 
코코로카라 
옥동식 서교점 
색동저고리 마포점 
마녀주방 홍대점 
일일향 마포점 
경기떡집 
뉴오더클럽 
어메이징농카이 
합정고깃집 
5  of  [  1  ] 
일등식당 
황곱 홍대점 
커피리브레 연남점 
몰토베네 
굴다리식당 
카쿠시타 
와이키키마켓 
톤앤매너 
다이닝원 상암점 
칸다소바 홍대점 
개화기요정 
1984 
피자네버슬립스 합정본점 
땡스오트 연남점 
램랜드 
1  of  [  2  ] 
아오이토리 
이치류 홍대본점 
가미우동 
카미야 
중화복춘 골드동교점 
빈브라더스 합정점 
스파카나폴리 
오랑지 
콜린 
부산집 
냉장고 
합정광안리 
장수갈매기 본점 
락희옥 마포본점 
합정옥 
2  of  [  2  ] 
여우골 홍대점 
이치젠